In [1]:
from datetime import datetime
import requests
import sys
import numpy as np
import pandas as pd

sys.path.append('..')
import futon

from bokeh.io import output_notebook, show, push_notebook
output_notebook()

Loading BokehJS ...

In [2]:
from futon.data.providers import Binance

# Add your developer API keys here
api_key = '<API KEY>'
secret_key = '<API SECRET>'

binance = Binance(api_key, secret_key)

In [3]:
coin = futon.instruments.Crypto('DOGE', 'USDT', provider = binance, interval = '30-min', start_date = '2021-05-01 00:00:00')

All caught up..!


In [4]:
coin.plot_candles()

In [5]:
from futon.strategy import TradingStrategy
from futon.indicators import Indicator, SimpleMovingAverage, ExponentialMovingAverage, BollingerBands

class MACDCrossover(TradingStrategy):
    def setup(self):
        self.macd = futon.indicators.MACD(fastperiod = 6, slowperiod = 18, signalperiod = 5, plot_separately = True)
        
        self.indicators = [self.macd]
    
    def logic(self, account, lookback):
        today = lookback.iloc[-1]

        macd_today, signal_today, _ = self.macd.lookback[-1]
        macd_yest, signal_yest, _ = self.macd.lookback[-2]

        # Buying
        buy_signal = (macd_today > signal_today) and (macd_yest < signal_yest)
        if buy_signal:
            entry_price   = today.close
            entry_capital = account.buying_power
            account.buy(entry_capital=entry_capital, entry_price=entry_price)

        # Selling
        else:
            sell_signal = (macd_today < signal_today) and (macd_yest > signal_yest)
            if sell_signal:
                percent = 1
                exit_price = today.close 
                account.sell(percent=percent, current_price=exit_price)

In [6]:
strat = MACDCrossover(coin)

In [7]:
strat.backtest(start_date = '2021-06-1 00:00:00', commision = 0.001, show_trades = True)

Performing backtest from: 01 June, 2021 (00:00:00) to 24 June, 2021 (19:30:00)



-------------- Results ----------------

Relative Returns: -17.83%
Relative Profit: -178.28

Strategy     : -39.8%
Net Profit   : -397.95

Buy and Hold : -21.97%
Net Profit   : -219.67

Buys        : 88
Sells        : 87
--------------------
Total Trades : 175

---------------------------------------
